# DuckDB Jaffle Shop Analysis Notebook

This notebook connects to the existing `jaffle.duckdb` file and performs core business analysis and visualization using Python libraries.

**Pre-requisite:** The file `jaffle.duckdb` with an `orders` table must be present in the same directory.

In [ ]:
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Set a consistent style for the visualizations
sns.set(style="whitegrid")

# Connect to the existing DuckDB database file
DB_FILE = 'jaffle.duckdb'
con = duckdb.connect(database=DB_FILE, read_only=True)
print(f"Successfully connected to {DB_FILE}")

## 1. Core Metrics & Status

Checking the total number of orders and their distribution by status.

In [ ]:
# Total Order Count
print("Total Orders:")
con.execute("SELECT COUNT(*) AS total_orders FROM orders").df()

In [ ]:
# Orders by Status (Bar Chart)
orders_status = con.execute("SELECT status, COUNT(*) AS num_orders FROM orders GROUP BY status").df()
plt.figure(figsize=(6,4))
sns.barplot(data=orders_status, x='status', y='num_orders', palette='viridis')
plt.title('Orders by Status')
plt.xlabel('Order Status')
plt.ylabel('Number of Orders')
plt.tight_layout()
plt.show()

## 2. Revenue & Value Metrics

Analyzing monthly revenue trends and Average Order Value (AOV).

In [ ]:
# Monthly Revenue (Line Chart)
monthly_revenue = con.execute("""SELECT DATE_TRUNC('month', order_date) AS month, SUM(amount) AS revenue FROM orders GROUP BY month ORDER BY month""").df()
plt.figure(figsize=(10,5))
sns.lineplot(data=monthly_revenue, x='month', y='revenue', marker='o', color='darkorange')
plt.title('Monthly Revenue')
plt.xlabel('Month')
plt.ylabel('Total Revenue ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Revenue by Payment Type (Pie Chart)
payment_revenue = con.execute("SELECT SUM(credit_card_amount) AS credit_card, SUM(bank_transfer_amount) AS bank_transfer, SUM(gift_card_amount) AS gift_card, SUM(coupon_amount) AS discount FROM orders").df()

plt.figure(figsize=(7,7))
colors = sns.color_palette('pastel')[0:4]
plt.pie(
    payment_revenue.iloc[0],
    labels=payment_revenue.columns,
    autopct='%1.1f%%',
    startangle=90,
    wedgeprops={'edgecolor': 'white'},
    colors=colors
)
plt.title('Revenue by Payment Type')
plt.show()

In [ ]:
# Average Order Value (AOV) by Month (Line Chart)
aov_month = con.execute("SELECT DATE_TRUNC('month', order_date) AS month, AVG(amount) AS avg_order_value FROM orders GROUP BY month ORDER BY month").df()
plt.figure(figsize=(10,5))
sns.lineplot(data=aov_month, x='month', y='avg_order_value', marker='o', color='forestgreen')
plt.title('Average Order Value (AOV) by Month')
plt.xlabel('Month')
plt.ylabel('Average Order Value ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 3. Customer Analysis

Identifying customer behavior, including repeat purchases and high-value customers.

In [ ]:
# Repeat Customer Analysis
repeat_customers = con.execute("""WITH customer_orders AS (SELECT customer_id, COUNT(order_id) AS num_orders FROM orders GROUP BY customer_id) SELECT SUM(CASE WHEN num_orders > 1 THEN 1 ELSE 0 END) AS repeat_customers, COUNT(*) AS total_customers, 100.0 * SUM(CASE WHEN num_orders > 1 THEN 1 ELSE 0 END)/COUNT(*) AS repeat_percent FROM customer_orders""").df()
print("Repeat Customer Metrics:")
repeat_customers

In [ ]:
# Top 10 Customers by Total Spend (Bar Chart)
top_customers = con.execute("SELECT customer_id, SUM(amount) AS total_spent FROM orders GROUP BY customer_id ORDER BY total_spent DESC LIMIT 10").df()
plt.figure(figsize=(10,5))
sns.barplot(data=top_customers, x='customer_id', y='total_spent', palette='rocket')
plt.title('Top 10 Customers by Total Spend')
plt.xlabel('Customer ID')
plt.ylabel('Total Spent ($)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Cleanup

Close the database connection to release the file lock.

In [ ]:
# Close the DuckDB connection
con.close()
print("DuckDB connection closed.")